<a href="https://colab.research.google.com/github/mariliaoliv/Computacao-Evolutiva/blob/main/eightqueens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* População grande (200) acelera bastante a convergência quando combinada com uma taxa de mutação mais alta (0.15).

* Taxas de mutação muito baixas (< 0.05) em populações pequenas não são eficazes.

* A combinação Pop=200 e Mut=0.15 é a mais eficiente no conjunto.

* Taxa de mutação adaptativa (com aumento em caso de estagnação) ajuda, mas só quando combinada com um espaço genético suficientemente diverso (população grande).

* Alta mutação com população pequena pode introduzir ruído demais (ex: Pop=50, Mut=0.1 teve só 12.5% de sucesso).

* O tempo de execução cresce quase linearmente com o tamanho da população, mas o ganho em convergência compensa.

In [7]:
import random
import time
from collections import defaultdict

# Meu
# Configurações
POPULATION_SIZES = [50, 100, 150, 200]
MUTATION_RATES = [0.01, 0.05, 0.08, 0.1, 0.15]
N_RUNS = 8
MAX_GENERATIONS = 1000
TARGET_FITNESS = 28

# Funções auxiliares
def create_individual():
    return [random.randint(0, 7) for _ in range(8)]

def evaluate_fitness(genes):
    clashes = 0
    for i in range(8):
        for j in range(i + 1, 8):
            if genes[i] == genes[j] or abs(genes[i] - genes[j]) == abs(i - j):
                clashes += 1
    return 28 - clashes

def create_population(pop_size):
    return sorted([(create_individual(), evaluate_fitness(create_individual()))
                  for _ in range(pop_size)], key=lambda x: x[1], reverse=True)

def select_parent(population, tournament_size=5):
    candidates = random.sample(population, tournament_size)
    return max(candidates, key=lambda x: x[1])

def crossover(parent1, parent2):
    point1, point2 = sorted(random.sample(range(1, 8), 2))
    child = parent1[0][:point1] + parent2[0][point1:point2] + parent1[0][point2:]
    return (child, evaluate_fitness(child))

def adaptive_mutation(individual, mutation_rate, generation, last_improvement):
    if generation - last_improvement > 50:
        mutation_rate = min(0.3, mutation_rate * 1.2)
    genes, fitness = individual
    if random.random() < mutation_rate:
        genes[random.randint(0, 7)] = random.randint(0, 7)
    return (genes, evaluate_fitness(genes)), mutation_rate

def evolve_population(population, mutation_rate, generation, last_improvement):
    new_pop = population[:20]
    for _ in range(len(population) - 20):
        parent1 = select_parent(population)
        parent2 = select_parent(population)
        child = crossover(parent1, parent2)
        child, mutation_rate = adaptive_mutation(child, mutation_rate, generation, last_improvement)
        new_pop.append(child)
    new_pop.sort(key=lambda x: x[1], reverse=True)
    return new_pop, mutation_rate

# Teste automatizado
def run_optimized_test():
    results = defaultdict(list)
    for pop_size in POPULATION_SIZES:
        for mut_rate in MUTATION_RATES:
            print(f"\n► Testando Pop={pop_size}, Mut={mut_rate}...")
            success_count = 0
            for _ in range(N_RUNS):
                pop = create_population(pop_size)
                best_fitness = pop[0][1]
                last_improvement = 0
                current_mut_rate = mut_rate
                start_time = time.time()
                for generation in range(MAX_GENERATIONS):
                    if best_fitness == TARGET_FITNESS:
                        success_count += 1
                        break
                    pop, current_mut_rate = evolve_population(pop, current_mut_rate, generation, last_improvement)
                    new_best = pop[0][1]
                    if new_best > best_fitness:
                        best_fitness = new_best
                        last_improvement = generation
                exec_time = time.time() - start_time
                results[(pop_size, mut_rate)].append((generation, exec_time, best_fitness))
            success_rate = (success_count / N_RUNS) * 100
            avg_gens = sum(r[0] for r in results[(pop_size, mut_rate)]) / N_RUNS
            avg_time = sum(r[1] for r in results[(pop_size, mut_rate)]) / N_RUNS
            print(f"  ✓ Sucesso: {success_rate}% | Gerações: {avg_gens:.1f} | Tempo: {avg_time:.2f}s")
    return results

if __name__ == "__main__":
    final_results = run_optimized_test()




► Testando Pop=50, Mut=0.01...
  ✓ Sucesso: 25.0% | Gerações: 773.9 | Tempo: 0.51s

► Testando Pop=50, Mut=0.05...
  ✓ Sucesso: 37.5% | Gerações: 738.9 | Tempo: 0.48s

► Testando Pop=50, Mut=0.08...
  ✓ Sucesso: 50.0% | Gerações: 595.9 | Tempo: 0.48s

► Testando Pop=50, Mut=0.1...
  ✓ Sucesso: 12.5% | Gerações: 892.8 | Tempo: 0.64s

► Testando Pop=50, Mut=0.15...
  ✓ Sucesso: 50.0% | Gerações: 550.4 | Tempo: 0.36s

► Testando Pop=100, Mut=0.01...
  ✓ Sucesso: 50.0% | Gerações: 612.4 | Tempo: 1.21s

► Testando Pop=100, Mut=0.05...
  ✓ Sucesso: 75.0% | Gerações: 272.8 | Tempo: 0.47s

► Testando Pop=100, Mut=0.08...
  ✓ Sucesso: 25.0% | Gerações: 764.0 | Tempo: 1.47s

► Testando Pop=100, Mut=0.1...
  ✓ Sucesso: 12.5% | Gerações: 956.1 | Tempo: 1.80s

► Testando Pop=100, Mut=0.15...
  ✓ Sucesso: 50.0% | Gerações: 513.2 | Tempo: 1.04s

► Testando Pop=150, Mut=0.01...
  ✓ Sucesso: 50.0% | Gerações: 510.9 | Tempo: 1.60s

► Testando Pop=150, Mut=0.05...
  ✓ Sucesso: 87.5% | Gerações: 251.1 | 